In [ ]:
! pip install transformers datasets peft 
! pip install evaluate
! pip install wandb
! pip install jiwer

In [ ]:
!pip install transformers==4.45.0


In [ ]:
import torch
print(torch.__version__)  # Shows PyTorch version
print(torch.version.cuda)  # Shows the CUDA version being used

In [ ]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [ ]:
import wandb

wandb.login()

In [ ]:
wandb.init(project="wav2vec2_fine_tune_single_speaker")

In [ ]:
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# Load pre-trained Wav2Vec2 model and processor
model_name = "ganga4364/mms_300_v4.96000"
model = Wav2Vec2ForCTC.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained(model_name)


In [ ]:
from datasets import load_from_disk

dataset_train = load_from_disk('/data/volume/wav2vec2/train_prepare_dataset.arrow')
dataset_val = load_from_disk('/data/volume/wav2vec2/val_prepare_dataset.arrow')

In [7]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [8]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [9]:
import evaluate

# Load the CER (Character Error Rate) metric
cer_metric = evaluate.load("cer")

In [10]:
import numpy as np 

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    print(pred_str)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    print(label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    print(cer)
    return {"cer": cer}

In [19]:
training_args = TrainingArguments(
        output_dir="./specific_speaker_model",
        evaluation_strategy="steps",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=2,
        learning_rate=3e-4,
        num_train_epochs=100,
        warmup_steps=500,
        logging_steps=100,
        eval_steps=1000,
        save_steps=1000,
        fp16=True,
        save_total_limit=50,
        dataloader_num_workers=4,
    )



In [20]:
model.freeze_feature_encoder()

In [21]:
model.config.ctc_zero_infinity = True

In [28]:
  # Create the Trainer object
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=processor.feature_extractor
)

In [ ]:

# Fine-tune the model
trainer.train()
#trainer.train(resume_from_checkpoint=True)


In [ ]:
# Save the fine-tuned model
print("training finished")